In [22]:
import json
from typing import NamedTuple


from kfp import dsl
from kfp.v2 import compiler
from typing import NamedTuple
from kfp.v2 import dsl
from kfp.v2.dsl import (
    component,
    InputPath,
    OutputPath,
    Input,
    Output,
    Artifact,
    Dataset,
    Model,
    ClassificationMetrics,
    Metrics,
)

from kfp.v2.google.client import AIPlatformClient
from view_demo.utils import get_project_id
from google_cloud_pipeline_components import aiplatform as gcc_aip

In [23]:
get_project_id()

'pytorch-tpu-nfs'

In [24]:
from view_demo.preprocess import create_dataset
a = create_dataset(
    csv_path='gs://bench-datasets/jena_climate_2009_2016.csv'
)

Client creating using default project: pytorch-tpu-nfs
Dataset view_dataset already exists
Loaded dataframe to /projects/pytorch-tpu-nfs/datasets/view_dataset/tables/weather_time_series


In [27]:
PROJECT_ID = get_project_id()
REGION = 'us-central1'
CONTAINER_URI = 'gcr.io/pytorch-tpu-nfs/test-custom-container'
SRC_ROOT = '..'
PIPELINE_ROOT = 'gs://automl-samples/pipelines/staging'

In [28]:
@component(
    base_image=CONTAINER_URI,
    output_component_file=f'{SRC_ROOT}/preprocess/preprocess.yaml',
)
def view_preprocess(
    project_id: str,
    raw_dataset: str,
    out_dataset: OutputPath(),
):
    from view_demo.preprocess import create_dataset
    bq_path = create_dataset(project_id=project_id, csv_path=raw_dataset)
    with open(out_dataset, 'w') as f:
        f.write(bq_path)
    

In [29]:
@dsl.pipeline(
    name="view-test-pipeline",
    pipeline_root=PIPELINE_ROOT,
)
def view_pipeline(
    project_id: str = PROJECT_ID,
    raw_dataset: str = 'gs://bench-datasets/jena_climate_2009_2016.csv',
):
    import logging
    preprocess_task = view_preprocess(project_id=project_id, raw_dataset=raw_dataset)


In [30]:
from kfp.v2 import compiler as v2compiler
v2compiler.Compiler().compile(pipeline_func=view_pipeline,
                              package_path='view_pipeline_spec.json')

In [31]:
from kfp.v2.google.client import AIPlatformClient  # noqa: F811

api_client = AIPlatformClient(
    project_id=PROJECT_ID, 
    region=REGION, 
    )

In [32]:
result = api_client.create_run_from_job_spec(
    job_spec_path="view_pipeline_spec.json",
    pipeline_root=PIPELINE_ROOT,
    enable_caching=False,
    parameter_values={
    },
)